In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import seaborn as sns

In [ ]:
# load visual inspection
dirname = '/home/alberto/almacen/Selected_LAEs/obs_mock_vi/'
# vi_path_name = f'{dirname}/ALTEROS_VI_250.txt'
vi_path_name = f'{dirname}/20230222_classification_dani_first.txt'
sgl = pd.read_csv(vi_path_name, sep=' ', header=None)
sgl[1]

In [ ]:
# Load real values
id_arr_to_plot = np.load(f'{dirname}/id_arr_to_plot.npy')
shuffle_indices = np.load(f'{dirname}/shuffle_indices.npy')
with open(f'{dirname}/mock_sel_dict.pkl', 'rb') as f:
    mock_sel_dict = pickle.load(f)

mock_class = np.ones(len(sgl)).astype(int) * -1 # -1 means real source
class_dict = {
    'LAE_QSO': 1,
    'CONT_QSO': 2,
    'GAL': 3,
    'SFG': 5
}
for i in range(128):
    src = shuffle_indices[i]
    cl = mock_sel_dict['class'][id_arr_to_plot[np.where(shuffle_indices == src)]][0]
    mock_class[src] = class_dict[cl]

In [ ]:
is_real = np.ones(len(sgl)).astype(bool)
is_real[shuffle_indices[:128]] = False

In [ ]:
is_real_sgl = sgl[2] == 'r'

In [ ]:
# print(f'Mock or real guess: {sum(is_real_sgl == is_real)}/{len(sgl)}')

# Confussion matrix
cf_mat = np.zeros((3, 4))
for reality in range(3):
    for guess in range(4):
        cf_mat[reality, guess] = sum((sgl[1] == guess + 1) & (mock_class == reality + 1))

fig, ax = plt.subplots()

sns.heatmap(cf_mat, ax=ax, annot=True, cbar=False)

ax.set_ylabel('REAL class', fontsize=18)
ax.set_xlabel('SID\'s VI class', fontsize=18)

ax.set_yticklabels(['LAE_QSO', 'CONT_QSO', 'GAL'])
ax.set_xticklabels(['LAE_QSO', 'CONT_QSO', 'GAL', 'IDK'])

plt.show()